In [2]:
import sys
!{sys.executable} -m pip install storysniffer
!{sys.executable} -m pip install newspaper3k
!{sys.executable} -m pip install -U dill
!{sys.executable} -m pip install scikit-learn==1.0.2

     |████████████████████████████████| 2.9MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 9.0MB/s eta 0:00:01
     |████████████████████████████████| 215kB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 7.4MB 11.9MB/s eta 0:00:01
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp37-none-any.whl size=13537 sha256=4b35160e9bc0c4bbe9c6f5fb838b13de948f1b2527dbc7b6dff63bf6f62a6862
  Stored in directory: /home/mazz76/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp37-none-any.whl size=3356 sha256=66a7074f76b8fb52a23896ed0966462b8a4b0338ac9e1538c0552a98cee1ef88
  Stored in directory: /home/mazz76/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp37-none-any.whl size=7398405 sha256=eba65c6ad88e3c515492f2fff770b09ca8b41159a0970c1bb1e91e45dd675a8a
  Stored in directory: /home

     |████████████████████████████████| 112kB 1.5MB/s eta 0:00:01
  Found existing installation: dill 0.2.9
    Uninstalling dill-0.2.9:
      Successfully uninstalled dill-0.2.9
     |████████████████████████████████| 23.0MB 3.5MB/s eta 0:00:01
  Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [1]:
import requests
from urllib.parse import urlparse, urljoin
import sys
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json
import urllib.parse
import hashlib
from storysniffer import StorySniffer
import ast
from newspaper import Article
import nltk
import pandas as pd

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_news_urls(url):
    
    # all URLs of `url` 
    domain_name = urlparse(url).netloc
    urls = set()
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko; Google Page Speed Insights) Chrome/41.0.2272.118 Safari/537.36'})
    soup = BeautifulSoup(response.content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                external_urls.add(href)
            continue
        urls.add(href)
        internal_urls.add(href)      
        
def crawl(news_url):
    print('get link ==> ', news_url)
    resp = requests.get(news_url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko; Google Page Speed Insights) Chrome/41.0.2272.118 Safari/537.36'})
    return resp.text

def get_host(url: str):
    token = url.split('://')[1]
    token = token.split('.')
    if token[0] == 'www':
        return token[1]
    return token[0]

In [2]:
internal_urls = set()
external_urls = set()
    
url = 'http://moscout.com/'

urls = get_news_urls(url)

valid_urls = []
    
sniffer = StorySniffer()

for u in internal_urls:
    try:
        if ".pdf" in u: 
            continue
        if "/event" in u: 
            continue
        if "/places" in u: 
            continue
        if "/classifieds" in u:
            continue
        if "/special_sections" in u:
            continue
        if "/form" in u:
            continue
        if "/edition" in u:
            continue
        if "/archive" in u:
            continue
        if "/feed" in u:
            continue
        if "gallery" in u:
            continue
        if "/ads" in u:
            continue
        if "/promo" in u:
            continue
        if "/video" in u:
            continue
        if "/slideshow" in u:
            continue
        if "/members-only" in u:
            continue
#Video only series from the STL Post-Dispatch
        if "/the-bottom-line" in u:
            continue
#Removing non-article pages from the Suburban Journals of Greater St. Louis
        if (urllib.parse.urlsplit(u).hostname == 'www.stltoday.com') and (not '/article' in u):
            continue
#Removing the landing pages from the St. Louis Review
        if (urllib.parse.urlsplit(u).hostname == 'www.archstl.org') and (not u[-4:].isnumeric()):
            continue
#Removing landing pages from the Sullivan Independent News
        if (urllib.parse.urlsplit(u).hostname == 'www.mysullivannews.com') and ("bourbon" in u):
            continue
#Removing landing pages from The Fulton Sun
        if (urllib.parse.urlsplit(u).hostname == 'www.fultonsun.com') and (('/fulton-news/' in u) or (not '-' in u)):
            continue
#Removing image only pages from The Joplin Globe
        if (urllib.parse.urlsplit(u).hostname == 'www.joplinglobe.com') and ('/image' in u):
            continue
#Removing calendar pages from The Marshall Democrat-News
        if (urllib.parse.urlsplit(u).hostname == 'www.marshallnews.com') and ('-calendar' in u):
            continue
#Removing landing pages from the Maryville Forum
        if (urllib.parse.urlsplit(u).hostname == 'www.maryvilleforum.com') and (('/multimedia/photos' in u) or (not '.html' in u)):
            continue
#Removing landing pages from The Pitch
        if (urllib.parse.urlsplit(u).hostname == 'www.thepitchkc.com') and (('/city-rag-kansas-city-news' in u) or ('/dessert-week-2022' in u)):
            continue
#Removing advertising pages from The Standard
        if (urllib.parse.urlsplit(u).hostname == 'www.the-standard.org') and ('/advertise' in u):
            continue
#Removing landing pages from the Webster Kirkwood News
        if (urllib.parse.urlsplit(u).hostname == 'websterkirkwoodtimes.com') and (not '/article' in u):
            continue
#Removing landing pages from the West End Word
        if (urllib.parse.urlsplit(u).hostname == 'www.timesnewspapers.com') and (not '/article' in u):
            continue
#Removing landing pages from KRZK
        if (urllib.parse.urlsplit(u).hostname == 'www.legends1063.fm') and ('/advertise-with-us' in u):
            continue
#Removing landing pages from KCUR and KSMU
        if (urllib.parse.urlsplit(u).hostname in ['www.kcur.org', 'www.ksmu.org', 'www.krcu.org']) and (re.findall(r'\d{4}-\d{2}-\d{2}', u) == []):
            continue
#Removing landing pages from News Press Now
        if (urllib.parse.urlsplit(u).hostname == 'www.newspressnow.com') and (not '.html' in u):
            continue
#Removing landing pages from KTTS
        if (urllib.parse.urlsplit(u).hostname == 'www.ktts.com') and (not '-' in u):
            continue
#Removing landing pages from KPRS
        if (urllib.parse.urlsplit(u).hostname == 'www.kprs.com') and (re.findall(r'\d{4}', u) == []):
            continue
#Removing landing pages from My MO Info
        if (urllib.parse.urlsplit(u).hostname == 'www.mymoinfo.com') and ('/trading-post' in u):
            continue
#Removing landing pages from KBIA
        if (urllib.parse.urlsplit(u).hostname == 'www.kbia.org') and (('/show' in u) or ('/podcast' in u) or ('paul-pepper' in u) or (re.findall(r'\d{4}-\d{2}-\d{2}', u) == [])):
            continue
#Removing podcast landing pages from 97.1FM and 98.1FM
        if (urllib.parse.urlsplit(u).hostname == 'www.audacy.com') and (('/podcast' in u) or ('/author' in u)):
            continue
#Removing non-news from The Beat
        if (urllib.parse.urlsplit(u).hostname == 'www.digitalburg.com') and (not '/central-news' in u):
            continue
#Removing non-news from Farmer Publishing
        if (urllib.parse.urlsplit(u).hostname == 'farmerpublishing.com') and ('e-mail-newsletter' in u):
            continue
#Removing landing pages from Benton County Enterprise
        if (urllib.parse.urlsplit(u).hostname in ['www.bentoncountyenterprise.com']) and (not re.findall(r'\d{4}', u) == []):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['lewispnj.com', 'maconhomepress.com', 'www.theijnews.com']) and ('/articles' in u):
            valid_urls.append(u)
        if (urllib.parse.urlsplit(u).hostname == 'www.thecash-book.com') and (len(u)<55):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.hermannadvertisercourier.com', 'www.lakegazette.net']) and (re.findall(r'\d', u) == []):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['bocojo.com']) and (('/news' in u) or ('/obituaries' in u) or ('/sports' in u) or ('/opinion' in u)) and (not '/category' in u):
            valid_urls.append(u)    
        if (urllib.parse.urlsplit(u).hostname in ['dosmundos.com', 'newstalkkzrg.com', 'www.949kcmo.com', 'plattecountylandmark.com', 'www.missourinet.com', 'www.californiademocrat.com', 'www.kctv5.com', 'www.kspr.com']) and (re.findall(r'\d{4}\/', u) == []):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['www.redlatinastl.com']) and ('/noticia' in u):
            valid_urls.append(u)   
        if (urllib.parse.urlsplit(u).hostname in ['emissourian.com']) and (('/marketplace' in u) or (re.findall(r'\d', u) == [])):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['molawyersmedia.com', 'store.molawyersmedia.com']) and (('/submit' in u) or ('store.' in u) or (re.findall(r'\d{4}\/', u) == [])):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['carthagenewsonline.com']) and ('/series' in u):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['www.warrensburgstarjournal.com', 'kcroonews.com']) and (re.findall(r'\d{5}', u) == []):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['themissouritimes.com']) and (('quarterly-filings' in u) or ('quarterly-reports' in u)):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['www.westplainsdailyquill.net']) and (not '/stories' in u):
            continue 
        if (urllib.parse.urlsplit(u).hostname in ['www.lamardemocrat.com', 'monroecountyappeal.com', 'www.jamesporttricountyweekly.com', 'www.greenfieldvedette.com', 'www.shelbinaweekly.com']) and ('articles' in u):
            valid_urls.append(u) 
        if (urllib.parse.urlsplit(u).hostname in ['www.linncountyleader.com']) and ('/community-calendar' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['mdcp.nwaonline.com']) and (('/home' in u) or (not requests.get(u).ok)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['morgancountypress.com']) and ('students-of-the-month' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.ozarkcountytimes.com']) and (('real-ozarks' in u) or ('hootin-hollarin' in u) or ('ozark-county-cookin' in u) or ('times-past' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.pleasanthillview.com']) and (('ad-' in u) or ('hidden-news' in u) or ('subscribe' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.waynecojournalbanner.com']) and ('election-results' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.vandalialeader.com']) and ('historical-society' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.warrencountyrecord.com']) and ('marketplace' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.plattechronicle.com']) and (('news' in u) or ('obituaries' in u) or ('sports' in u) or ('opinion' in u)):
            valid_urls.append(u)
        if (urllib.parse.urlsplit(u).hostname in ['www.gladstonedispatch.com']) and ('image' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.howellcountynews.com']) and ('local/' in u):
            valid_urls.append(u)
        if (urllib.parse.urlsplit(u).hostname in ['news-journal.net']) and ('date' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['metrovoicenews.com']) and (('about-us' in u) or ('community-events' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.417mag.com']) and (('people-pics' in u) or ('best-of-417' in u) or ('issues' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['moscout.com']) and ('2022-watch' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.kmov.com']) and (('news' in u) or ('page' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['fox2now.com']) and (('get-a-copy-of-a-newscast' in u) or ('closings' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.kmbc.com']) and ('news-tem' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.koamnewsnow.com']) and (('pledge-of-allegiance' in u) or ('road-condition-resources' in u) or ('joplin-news-first-on-koam' in u) or ('4-state-heroes-home' in u) or ('business-club' in u) or ('now-hiring-spotlight' in u) or ('4-states-in-depth' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.kfvs12.com']) and ('closed-captioning' in u):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['fox4kc.com']) and ((len(u)<60) or ('email-alerts' in u) or ('closed-captioning-contact-information' in u) or ('total-closings-sign-up' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.ozarksfirst.com', 'www.fourstateshomepage.com', 'www.kq2.com', 'www.mycouriertribune.com']) and (u.count('-')<6):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['abc17news.com']) and (('sign-up-for-our-newsletters' in u) or ('community-guidelines' in u) or ('back-to-school-headquarters' in u) or ('get-breaking-news-alerts' in u)):
            continue
        if (urllib.parse.urlsplit(u).hostname in ['www.komu.com']) and ('weather-maps' in u):
            continue
        
        if sniffer.guess(u) == True and (not u in valid_urls):
            valid_urls.append(u)

    except BaseException as e:
        pass
    except SSLError:
        pass
    except MaxRetryError:
        pass
    except SSLCertVerificationError:
        pass

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator FunctionTransformer from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own ris

In [ ]:
response = requests.get('https://kcroonews.com/')
response

In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [3]:
internal_urls

{'https://www.thekansascityglobe.com/',
 'https://www.thekansascityglobe.com/about-us',
 'https://www.thekansascityglobe.com/ada-compliance',
 'https://www.thekansascityglobe.com/advertise',
 'https://www.thekansascityglobe.com/e-edition/342',
 'https://www.thekansascityglobe.com/form/contact',
 'https://www.thekansascityglobe.com/form/news-tips',
 'https://www.thekansascityglobe.com/news',
 'https://www.thekansascityglobe.com/news-staff',
 'https://www.thekansascityglobe.com/news/battle-soul-america-hinges-nov-8-general-election',
 'https://www.thekansascityglobe.com/news/boulevard-brewing-vine-street-brewing-team-honor-buck-oneil-0',
 'https://www.thekansascityglobe.com/news/community-mourns-youth-sports-mentor-pastor-calvin-wainwright',
 'https://www.thekansascityglobe.com/news/council-passes-international-energy-conservation-code-standards-new-developments',
 'https://www.thekansascityglobe.com/news/jackson-county-prosecutors-office-responds-racist-threatening-voice-mail-messages',

In [3]:
valid_urls

['http://moscout.com/specialreports/2022/10/11/mec-fundy-for-earle-sears',
 'http://moscout.com/specialreports/2022/10/12/hruza-radio-ad',
 'http://moscout.com/specialreports/2022/11/8/anonymous-texts-in-house-98',
 'http://moscout.com/daily-updates-1/2022/11/8/moscout-daily-update-what-to-watch-for-tonight-late-hrcc-ies-andrews-gets-appt-dirty-tactics-in-house-98-and-more',
 'http://moscout.com/daily-updates-1/2022/11/7/moscout-daily-update-the-day-before-keonig-for-pro-tem-moscout-pickem-contest-appointment-talk-ballwin-fined-and-more',
 'http://moscout.com/daily-updates-1/2022/11/9/moscout-daily-update-picture-from-last-night-new-house-member-profiles-whats-next-and-more']

In [4]:
news_data = []
# print(valid_urls)
try:
    for u in valid_urls:
        hash = hashlib.sha256(u.encode())
        news_data.append({
            'id': hash.hexdigest(),
            'url': u,
            'crawl_date': str(datetime.now()),
            'text': crawl(u),
            'host': urllib.parse.urlsplit(u).hostname
            })
except BaseException as e:
    pass

get link ==>  http://moscout.com/specialreports/2022/10/11/mec-fundy-for-earle-sears
get link ==>  http://moscout.com/specialreports/2022/10/12/hruza-radio-ad
get link ==>  http://moscout.com/specialreports/2022/11/8/anonymous-texts-in-house-98
get link ==>  http://moscout.com/daily-updates-1/2022/11/8/moscout-daily-update-what-to-watch-for-tonight-late-hrcc-ies-andrews-gets-appt-dirty-tactics-in-house-98-and-more
get link ==>  http://moscout.com/daily-updates-1/2022/11/7/moscout-daily-update-the-day-before-keonig-for-pro-tem-moscout-pickem-contest-appointment-talk-ballwin-fined-and-more
get link ==>  http://moscout.com/daily-updates-1/2022/11/9/moscout-daily-update-picture-from-last-night-new-house-member-profiles-whats-next-and-more


In [ ]:
news_data

In [5]:
raw_articles = news_data

len(raw_articles)

6

In [17]:
article = raw_articles[0]


In [6]:
def given_tags(item,data):
    
    if data['hostname'] in ['www.bentoncountyenterprise.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("ul", {"class" : "links"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.examiner.net']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"id" : "breadcrumbs"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.semissourian.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("title")
            tags.append(results.get_text())
            return ' '.join(tags).split(':',1)[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.chillicothenews.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("a", {"rel" : "category tag"})
            tags.append(results.get_text())
            return tags[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.northwestmoinfo.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("nav", {"aria-label" : "Category Breadcrumbs"})
            tags.append(results.get_text())
            return ' '.join(tags).replace('\n','').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['fox2now.com', 'fox4kc.com', 'www.ozarksfirst.com', 'www.fourstateshomepage.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-eyebrow"})
            tags.append(results.get_text())
            return ' '.join(tags).replace('\n','').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.ksdk.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("a", {"class" : "article__category"})
            tags.append(results.get_text())
            return tags[0]
        
        except AttributeError:
            return ''
        
    if data['hostname'] in ['kcroonews.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("ul", {"class" : "snotags"})
            for x in results.find_all('li'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['carthagenewsonline.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("ul", {"class" : "td-category"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.missourinet.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"class" : "entry-tags"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['whatsupkansascity.net']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "rating-area"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['missouriindependent.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("ul", {"id" : "dataCats"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.417mag.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "headline"})
            results2=results.find('h5')
            if len(results2.get_text())<10:
                tags.append(results2.get_text())
            return ' '.join(tags).encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['metrovoicenews.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("p", {"class" : "post-tag"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.ozarkcountytimes.com', 'www.howellcountynews.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "field field-name-field-section field-type-taxonomy-term-reference field-label-inline clearfix"})
            for x in results.find_all('a'):
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['northeastnews.net']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:tag"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.northcassherald.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("li", {"class" : "TZ6_II"})
            y = results.find('a')
            tags.append(y.get_text())
            return tags[0].lower()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['morgancountypress.com', 'leader-statesman.com', 'eldoradospringsmo.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("li", {"class" : "c5-meta-li-category clearfix"})
            y = results.find_all('a')
            for x in y:
                tags.append(x.get_text())
            return tags
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.linncountyleader.com', 'www.southmissouriannews.com']:
        tags = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("a", {"rel" : "category tag"})
            tags.append(results.get_text())
            return ' '.join(tags)
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.thelickingnews.com', 'www.kahokamedia.com', 'www.themilanstandard.com', 
                           'www.palmyra-spectator.com', 'www.waynecojournalbanner.com', 'www.shelbycountyherald.com',
                           'www.stegenherald.com', 'www.memphisdemocrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "fl-post-cats-tags"})
            results2=results.find('a')
            auth.append(results2.get_text())
            return ' '.join(auth)
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.lawrencecountyrecord.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            results=soup.find("li", {"class" : "taxonomy-term-reference-0"})
            x = results.find_all('a')
            for y in x:
                tags.append(y.get_text().lower())
            return tags[0]
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.studlife.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            results=soup.find("p", {"class" : "article-tags"})
            x = results.find_all('a')
            for y in x:
                tags.append(y.get_text())
            return tags
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['plattecountylandmark.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("div", attrs={"class":"jeg_post_tags"})
        try:
            for x in key.find_all('a'):
                tags.append(x.get_text())
            return tags
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("span", attrs={"class":"tag-links"})
        try:
            for x in key.find_all('a'):
                tags.append(x.get_text())
            return tags
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.excelsiorspringsstandard.com', 'www.richmond-dailynews.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("ul", attrs={"class":"links field__items"})
        tag = key.find('a').get_text()
        tags.append(tag) 
        return tags[0]
    
    if data['hostname'] in ['www.dixonpilot.com', 'www.edinasentinel.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find("div", attrs={"class":"fl-post-cats-tags"})
        tag = key.find('a').get_text()
        tags.append(tag) 
        return [tags[0]]
    
    if data['hostname'] in ['missouri.statenews.net', 'www.kansascity.com', 'www.kxcv.org']:
        try:   
            tags = []
            soup = BeautifulSoup(article['text'],'html.parser')
            key = soup.find(attrs={"name":"keywords"})
            keyword = (key.get("content", None))
            tags.append(keyword) 
            return tags

        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.949kcmo.com':
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:section"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'unewsonline.com':
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"property":"article:section"})
            keyword = (key.get("content", None))
            tags.append(keyword) 
            return tags[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['molawyersmedia.com', 'www.pleasanthillview.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"keywords"})
            keyword = (key.get("content", None))
            keyword2 = keyword.split(",")
            tags=keyword2 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.hermannadvertisercourier.com', 'www.lakegazette.net', 'emissourian.com', 
                           'www.phelpscountyfocus.com', 'www.webstercountycitizen.com', 'www.gladstonedispatch.com',
                           'www.griffonnews.com', 'www.newstime-mo.com', 'www.nwmissourinews.com', 'www.hannibal.net',
                           'www.kmbc.com', 'www.kq2.com', 'www.komu.com', 'dailyjournalonline.com', 'www.lincolnnewsnow.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"news_keywords"})
            keyword = (key.get("content", None))
            keyword2 = keyword.split(",")
            tags=keyword2 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.lakenewsonline.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"name":"news_keywords"})
            keyword = (key.get("content", None))
            keyword2 = keyword.split(", ")
            tags=keyword2 
            return tags
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.audacy.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"name":"branch:deeplink:editorial_tags"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.kprs.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("a", attrs={"rel":"category tag"}).get_text()
            tags.append(key) 
            return tags[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
        
    if data['hostname'] in ['www.kcur.org', 'news.stlpublicradio.org']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"article:tag"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
        
    if data['hostname'] in ['www.krcu.org']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find_all("meta", attrs={"property":"article:tag"})
            for x in key:
                keyword = (x.get("content", None))
                tags.append(keyword) 
            return tags[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''

    if data['hostname'] in ['www.newspressnow.com', 'www.mycouriertribune.com']:
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("div", attrs={"class":"asset-tags hidden-print"})
            for x in key.find_all('a'):
                keyword = (x.get_text())
                tags.append(keyword) 
            return tags
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
    
    else: 
        pass

def given_location(item,data):
    if data['hostname'] == 'fox2now.com':
        loc = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find(attrs={"name":"primaryCategory"})
        # keyword = (key.get("content", None))
        loc.append(key)
        return loc
    

def given_date(item,data):
    
    if data['hostname'] in ['www.417mag.com']:
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "headline"})
            for x in results.find_all('p'):
                if len(x.get_text()) in range(9,35):
                    date.append(x.get_text())
            y = ' '.join(date).split('at',1)[0].strip()
            formatted = datetime.strptime(y, '%b %d %Y')
            z = (str(formatted))
            return z
        
        except AttributeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['kcroonews.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"class":"storydate"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['morgancountypress.com', 'leader-statesman.com', 'eldoradospringsmo.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("p", attrs={"class":"c5-today"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.redlatinastl.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("li", attrs={"class":"text-muted"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%Y-%m-%d')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['cedarrepublican.com', 'ccheadliner.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"class":"monthday"}).get_text().strip()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", attrs={"class":"col-12 col-sm contact-section"}).get_text().replace('\n','')
            txt_2 = datetime.strptime(txt, '%A, %B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.mycaldwellcounty.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"property":"dc:date dc:created"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['moberlymonitor.com', 'gasconadecountyrepublican.com', 'www.kirksvilledailyexpress.com', 
                            'www.warrensburgstarjournal.com', 'www.laclederecord.com', 'mexicoledger.com', 
                           'www.sedaliademocrat.com', 'www.westplainsdailyquill.net', 'mariescountyadvocate.com', 
                           'marshfieldmail.com', 'www.mystandardnews.com', 'republicmonitor.com', 'www.unterrifieddemocrat.com',
                           'www.warrencountyrecord.com', 'www.clintondailydemocrat.com', 'www.fayettenewspapers.com',
                           'bolivarmonews.com']:
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", {"name":"twt-published-at"})
            txt_date = txt.get("content", None)[:-6]
            #Tue, 11 Oct 2022 12:00:00
            txt_2 = datetime.strptime(txt_date, '%a, %d %b %Y %H:%M:%S')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] in ['www.kxcv.org']:
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", {"class":"date"})
            txt_date = txt.find('b').get_text().replace('Sept.','Sep').replace('Jan.','Jan').replace('Feb.','Feb').replace('March','Mar').replace('April','Apr').replace('June','Jun').replace('July','Jul').replace('Oct.','Oct').replace('Nov.','Nov').replace('Dec.', 'Dec')
            txt_2 = datetime.strptime(txt_date, '%b %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['www.theijnews.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"published-date"}).get_text().strip().replace(',','')
            #September 29, 2022
            txt_2 = datetime.strptime(txt, '%B %d %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['lewispnj.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"published-date "})
            #September 29, 2022
            #txt_2 = datetime.strptime(txt, '%B %d, %Y')
            #date.append(str(txt_2))
            date.append(txt.get_text().strip())
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['maconhomepress.com', 'www.lamardemocrat.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"published-date"}).get_text().strip()
            #September 29, 2022
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['www.firesideguard.com', 'www.vandalialeader.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"pss-dateline"}).get_text().replace('Sunday,','').replace('Monday,','').replace('Tuesday,','').replace('Wednesday,','').replace('Thursday,','').replace('Friday,','').replace('Saturday,','').strip().replace('th','').replace('nd','').replace('rd','').replace('st','')
            #October 9, 2022
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'www.legends1063.fm':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("p", {"class":"news-timestamp"}).get_text().strip().replace('th','').replace('nd','').replace('rd','').replace('st','').replace('am','AM').replace('pm','PM')
            #Thursday, September 22, 2022  6:21
            txt_2 = datetime.strptime(txt, '%A, %B %d, %Y  %I:%M%p')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] == 'www.4bcaonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            key = soup.find("meta", attrs={"property":"og:description"})
            date_text = (key.get("content", None))
            txt_2 = datetime.strptime(date_text, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''

    if data['hostname'] == 'unewsonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"time-wrapper"}).get_text()
            txt_2 = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'www.theccreporter.com':
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", string=re.compile(r', \d\d\d\d'))
            txt_cleaned = txt.get_text().replace('Published ','').replace('\n','').strip('\r')
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''   
    
    if data['hostname'] == 'themaneater.com':
        date = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"post-date"}).get_text()
            txt_cleaned = txt.replace('th','').replace('nd','').replace('rd','').replace('st','')
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.riverfronttimes.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"date"})
            date.append(txt["content"])
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.archstl.org':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"col-12 col-lg-3 ml-auto publish-date-meta text-right"}).get_text()
            txt_cleaned = txt.split('|', 1)[0].replace('Submitted','').replace('\r','').replace('\n','').strip()
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'callnewspapers.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"time-wrapper"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
        
    if data['hostname'] in ['www.bentoncountyenterprise.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", attrs={"property":"dc:date dc:created"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.plattecountycitizen.com', 'the-standard.org']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"datePublished"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'stlamerican.com',
                           'www.stltoday.com', 'www.joplinglobe.com', 'www.maryvilleforum.com', 'websterkirkwoodtimes.com',
                           'www.newspressnow.com', 'dailyjournalonline.com', 'www.hermannadvertisercourier.com',
                           'www.lakegazette.net', 'emissourian.com', 'www.phelpscountyfocus.com', 'www.webstercountycitizen.com',
                           'www.gladstonedispatch.com', 'www.griffonnews.com', 'www.newstime-mo.com', 'www.hannibal.net',
                           'www.kq2.com', 'www.komu.com', 'www.mycouriertribune.com', 'www.lakenewsonline.com', 
                           'www.lincolnnewsnow.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['todaysfarmermagazine.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"property":"datePublished"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
     
    if data['hostname'] == 'sgfneighborhoodnews.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"dtreviewed"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'thesalemnewsonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
   
    if data['hostname'] == 'www.columbiamissourian.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"parsely-pub-date"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'missouri.statenews.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        date.append(temp[1])
        return date
    
    elif data['hostname'] == 'www.kansascity.com':
        try:
            date = []
            soup = BeautifulSoup(article['text'],'html.parser')
            div = soup.find('time', attrs={'class':'update-date time'})
            srce = div['datetime']
            date.append(srce)
            y = ' '.join(date)
            txt_cleaned = pd.to_datetime(y).tz_convert('utc').tz_localize(None) 
            return str(txt_cleaned)
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        date = data['meta_data']['twt-published-at']
        return date
    
    if data['hostname'] == 'www.hannibal.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key  = soup.find('meta', attrs={'name': 'tncms-access-version'})
        content = key['content']
        date.append(content)
        return date
    
    else:
        
        date = data['date']
        return date
    

def source(item,data):
    if data['hostname'] == 'missouri.statenews.net': 
        src = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        src.append(temp[0])
        return src
    
    if data['hostname'] == 'abc17news.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__category'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src
    
    else: 
        
        pass


def author(item,data):
    
    if data['hostname'] in ['www.nevadadailymail.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"})
            auth.append(results.get_text())
            return ' '.join(auth).split('|',1)[0].replace('By','').replace('Photos by','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.thekansascityglobe.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"rel" : "schema:author"}).get_text()
            auth.append(results)
            auth2 = ' '.join(auth)
            return ' '.join(auth2.split()[:2])
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.semissourian.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"}).get_text()
            auth.append(results)
            return ' '.join(auth).replace('By','').strip()
        
        except AttributeError:
            return ''
        
    if data['hostname'] in ['www.lakenewsonline.com', 'www.lincolnnewsnow.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            if 'CNN' in txt["content"]:
                auth.append('CNN')
            elif 'AP' in txt["content"]:
                auth.append('The Associated Press')
            elif 'Celebretainment' in txt["content"]:
                auth.append('Celebretainment')
            else:
                auth.append(txt["content"])
            return ' '.join(auth).split(',',1)[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.myleaderpaper.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            if 'CNN' in txt["content"]:
                auth.append('CNN')
            elif 'Columbia Missourian' in txt["content"]:
                auth.append('Columbia Missourian')
            elif 'AP' in txt["content"]:
                auth.append('The Associated Press')
            else:
                auth.append(txt["content"])
            return ' '.join(auth).split(',',1)[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.kq2.com', 'www.komu.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            if 'CNN' in txt["content"]:
                auth.append('CNN')
            elif 'Columbia Missourian' in txt["content"]:
                auth.append('Columbia Missourian')
            elif 'AP' in txt["content"]:
                auth.append('The Associated Press')
            else:
                auth.append(txt["content"])
            return ' '.join(auth).split(',',1)[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['abc17news.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find("div", attrs={"class":"meta__user vcard author"})
            for x in results.find_all('a'):
                auth.append(x.get_text())
            return auth

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.fourstateshomepage.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find("p", attrs={"class":"article-authors"})
            auth.append(results.get_text())
            return ' '.join(auth).replace('\t','').replace('\n','').replace('by:','').strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.ozarksfirst.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find("p", attrs={"class":"article-authors"})
            for x in results.find_all('a'):
                auth.append(x.get_text())
                if auth == []:
                    for y in results.find_all('span'):
                        auth.append(y.get_text())
            return auth

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.kmbc.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find("div", attrs={"class":"article-byline--details-author"})
            for x in results.find_all('a'):
                if "CNN" in x.get_text():
                    auth.append('CNN')
                    continue
                if "AP" in x.get_text():
                    auth.append('The Associated Press')
                    continue
                auth.append(x.get_text())
            return ' '.join(auth).replace('\t','').replace('\n','').strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['fox2now.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find("p", attrs={"class":"article-authors"})
            for x in results.find_all('a'):
                auth.append(x.get_text())
            return auth

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.ksdk.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("article", attrs={"class":"article"})
            auth.append(txt["data-author"])
            x = ' '.join(auth).strip() 
            y = x.split(", ")
            auth=y 
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.kmov.com', 'www.kctv5.com', 'www.kspr.com', 'www.kfvs12.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"class" : "author |"}).get_text()
            auth.append(results)
            return auth[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.417mag.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "headline"})
            results2=results.find('h3')
            auth.append(results2.get_text())
            return ' '.join(auth).encode('ascii', 'ignore').decode('ascii').replace('by', '').replace('By', '').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['metrovoicenews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "entry"})
            results2 = results.find('em')
            auth.append(results2.get_text())
            return ' '.join(auth).encode('ascii', 'ignore').decode('ascii').split('|',1)[0].replace('-','').strip()
            
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.greenfieldvedette.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "author"})
            auth.append(results.get_text())
            return ' '.join(auth).replace('by','').split(',',1)[0].strip()
            
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.vandalialeader.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "content"})
            for p in results.find_all('p'):
                if p.get_text().startswith('By'):
                    auth.append(p.get_text())
            return ' '.join(auth).replace('By','').strip()
            
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.southmissouriannews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"})
            auth.append(results.get_text())
            return auth[0]
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.pulaskicountydaily.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"class" : "page_byline"})
            auth.append(results.get_text())
            return ' '.join(auth).replace('By:','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.ozarkcountytimes.com', 'www.howellcountynews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "field field-name-field-dateline field-type-text field-label-hidden"})
            auth.append(results.get_text())
            return ' '.join(auth).split(',',1)[0].replace('By','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['northeastnews.net']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("a", {"rel" : "author"})
            auth.append(results.get_text())
            return ' '.join(auth)
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['morgancountypress.com', 'leader-statesman.com', 'eldoradospringsmo.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-padding clearfix"})
            for p in results.find_all('p'):
                if p.get_text().lower().startswith('by') and len(p.get_text())<100:
                    auth.append(p.get_text())
            return ' '.join(auth).split(',', 1)[0].replace('by','').replace(':','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
        
    if data['hostname'] in ['republicmonitor.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"})
            auth.append(results.get_text())
            return ' '.join(auth).replace('By','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
        
    if data['hostname'] in ['www.theprospectnews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"})
            auth.append(results.get_text())
            return ' '.join(auth).replace('By','').split('Sports',1)[0].split('Staff',1)[0].strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['standard-democrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "byline"})
            auth.append(results.get_text())
            return ' '.join(auth).split('/',1)[0].replace('By','').strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.themilanstandard.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            for p in results.find_all('p'):
                if p.get_text().startswith('Submitted'):
                    auth.append(p.get_text())
            return ' '.join(auth).split('by',1)[1].split(',',1)[0].strip()
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['www.kahokamedia.com', 'www.palmyra-spectator.com', 'www.waynecojournalbanner.com', 'www.memphisdemocrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"itemprop" : "name"})
            auth.append(results.get_text())
            return auth[0]
        
        except AttributeError:
            return ''  
    
    if data['hostname'] in ['mdcp.nwaonline.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"itemprop":"author"})
            auth.append(txt["content"])
            return ' '.join(auth).split('/',1)[0]    

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.thelickingnews.com', 'www.shelbycountyherald.com', 'www.stegenherald.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            for p in results.find_all('p'):
                if p.get_text().startswith('By'):
                    auth.append(p.get_text())
            return ' '.join(auth).split(',',1)[0].replace('By','').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.lawrencecountyrecord.com', 'www.bentoncountyenterprise.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"field field-name-field-by field-type-text field-label-inline clearfix"})
            txt2 = txt.get_text()
            auth.append(txt2)
            return ' '.join(auth).encode('ascii', 'ignore').decode('ascii').replace('By:','').replace('Submitted by','').strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
        
    if data['hostname'] in ['missouriindependent.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"byline"})
            txt2 = txt.get_text()
            auth.append(txt2)
            return ' '.join(auth).replace('by','').split(',',1)[0].strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['www.kansascity.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"byline"})
            txt2 = txt.get_text()
            auth.append(txt2)
            return ' '.join(auth).replace('By','').strip().split('\n',1)[0].replace('AND','and').split('and',1)[0].strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.warrensburgstarjournal.com', 'www.sedaliademocrat.com', 'www.westplainsdailyquill.net',
                           'www.fayettenewspapers.com', 'bolivarmonews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split(",",1)[0].replace('By','').replace('BY', '').split('\n',1)[0].strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['www.studlife.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("span", {"class" : "author-name"})
            x = results.find_all('a')
            for y in x:
                auth.append(y.get_text())
            return auth
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['bocojo.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-white-box layout-2 c5-margin clearfix"})
            x = results.find_all('p')
            for y in x:
                if y.get_text().startswith('By:'):
                    auth.append(y.get_text())
            return ' '.join(auth).replace('By:','').strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''

    if data['hostname'] in ['ccheadliner.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).replace('By','').strip().split("\n",1)[0].lower()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.laclederecord.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').replace(':','').strip().lower()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
        
    if data['hostname'] in ['mexicoledger.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split(',',1)[0].replace('By','').replace('Photos by','').replace('story by','').strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
        
    if data['hostname'] in ['mariescountyadvocate.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').replace('BY','').split(',',1)[0].strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['marshfieldmail.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').strip()
            z = re.sub(r'(([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+))', '', txt2)
            auth.append(z)
            return ' '.join(auth).strip()
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.unterrifieddemocrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').split(',',1)[0].strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.warrencountyrecord.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').split(',',1)[0].strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
        
    if data['hostname'] in ['moberlymonitor.com', 'cedarrepublican.com', 'gasconadecountyrepublican.com', 
                            'www.kirksvilledailyexpress.com', 'www.mystandardnews.com', 'www.clintondailydemocrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"twt-author-name"})
            txt2 = (txt.get("content", None)).split('\n',1)[0].replace('By','').strip()
            auth.append(txt2)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['maconhomepress.com', 'www.lamardemocrat.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"author"})
            x = txt.get_text().replace('by','').strip()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.mycaldwellcounty.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"property":"content:encoded"})
            x = txt.find('a').get_text()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.excelsiorspringsstandard.com', 'www.richmond-dailynews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"user-name"})
            x = txt.find('a').get_text().split('And',1)[0].strip()
            auth.append(x)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.dixonpilot.com', 'www.edinasentinel.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", attrs={"itemprop":"name"}).get_text()
            auth.append(txt)
            return auth[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return '' 
    
    if data['hostname'] in ['www.kxcv.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("i").get_text().encode('ascii', 'ignore').decode('ascii')
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.kprs.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", attrs={"class":"single-meta"}).get_text()
            txt2 = txt.replace('|','').strip()
            auth.append(txt2)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return '' 

    if data['hostname'] in ['www.kbia.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"gtm-dataLayer"})
            txt2 = txt["content"].strip()
            txt_dict = json.loads(txt2)
            auth.append(txt_dict['gtmAuthor'].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''  
    
    if data['hostname'] in ['www.kcur.org', 'news.stlpublicradio.org', 'www.ksmu.org', 'www.krcu.org']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"gtm-dataLayer"})
            txt2 = txt["content"].strip()
            txt_dict = json.loads(txt2)
            auth.append(txt_dict['gtmAuthor'].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''   
    
    
    if data['hostname'] == 'www.legends1063.fm':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"news-author"})
            auth.append(txt.get_text().replace('By','').replace('\n','').strip().split('  ', 1)[0])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'themaneater.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("p", attrs={"class":"post-author"})
            auth.append(txt.get_text().strip('By '))
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.southeastarrow.com', 'www.nevadadailymail.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"byline"})
            auth.append(txt.get_text())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.archstl.org':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        #selector = 'div.col-8 > h3'
        try:
            txt = soup.find("div", attrs={"class":"author-meta d-flex flex-wrap justify-content-center flex-lg-column justify-content-lg-start"})
            auth.append(txt.get_text().replace('\n','').replace('\r','').strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.news-leader.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"property":"article:author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.semissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        try:
            txt = soup.find('a', {'class':['columnistbyline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'sgfneighborhoodnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.gladstonedispatch.com', 'www.griffonnews.com','thesalemnewsonline.com', 'stlamerican.com', 'www.pikecountynews.com', 
                            'newspressnow.com', 'www.newspressnow.com', 'callnewspapers.com', 'www.stltoday.com', 'www.riverfronttimes.com',
                           'www.joplinglobe.com', 'websterjournal.com', 'plattecountylandmark.com', 
                           'www.maryvilleforum.com', 'the-standard.org', 'unewsonline.com', 'websterkirkwoodtimes.com', 
                           'www.thecash-book.com', 'www.hermannadvertisercourier.com',
                           'www.lakegazette.net', 'emissourian.com', 'molawyersmedia.com', 'themissouritimes.com', 
                           'www.phelpscountyfocus.com', 'www.pleasanthillview.com', 'www.newstime-mo.com', 
                           'www.nwmissourinews.com', 'todaysfarmermagazine.com', 'www.hannibal.net', 'www.mycouriertribune.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"].split(',',1)[0].replace('Story by','').replace('By', '').split('|',1)[0].replace('Compiled by','').replace('/Courier-Tribune','').strip())
            auth2 = ' '.join(auth)
            return ' '.join(auth2.split()[:2])

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['dailyjournalonline.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            if "St. Louis Post-Dispatch" in txt["content"]:
                auth.append('St. Louis Post Dispatch')
            elif "Lee Affiliate" in txt["content"]:
                auth.append('Lee Affiliate')
            elif "Sponsored content" in txt["content"]:
                auth.append('Sponsored Content')
            elif "Associated Press" in txt["content"]:
                auth.append('The Associated Press')
            else:
                auth.append(txt["content"])
                auth[0] = re.sub(r'(([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+))', '', auth[0])
            return ' '.join(auth).replace('Lee Enterprises','').split(',',1)[0].strip()

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.darnews.com', 'www.dddnews.com', 'www.dexterstatesman.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", attrs={"class":"byline"})
            auth.append(txt.get_text())
            auth2 = ' '.join(auth).replace('By', '').replace(',','').strip()
            return ' '.join(auth2.split()[:2])

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'lstribune.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt =  soup.find("strong").get_text()
            if txt.startswith('By'):
                auth.append(txt)
                return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    
    if data['hostname'] in ['www.newstribune.com', 'www.fultonsun.com', 'www.plattecountycitizen.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"itemprop":"author"})
            auth.append(txt["content"])
            return auth[0]    

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.greenecountycommonwealth.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', class_='field-item even').get_text()
            if len(txt) <50:
                auth.append(txt)
                return auth[0]
            else: 
                return ''
            

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    

    if data['hostname'] == 'www.columbiamissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'www.audacy.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", property="article:author:name")
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.columbiatribune.com', 'newstalkkzrg.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", property="article:author")
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
       
    
    if data['hostname'] == 'mycameronnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'field field-name-field-dateline field-type-text field-label-inline clearfix'}).get_text().strip('By:\xa0')
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.cassville-democrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'carthagenewsonline.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", {"class": "td-post-author-name"})
            anchor = txt.find('a').get_text()
            auth.append(anchor)
            return auth[0]

        except AttributeError:
            return ''

    if data['hostname'] == 'www.californiademocrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('a', attrs={'class':'bi-line-link'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'auroraadvertiser.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        spans = soup.find('span', attrs={'class':'byline'})
        auth.append(spans.text)
    
    if data['hostname'] == 'www.webstercountycitizen.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", { "itemprop" : "author" }).get_text()
            auth.append(txt)
            return ' '.join(auth).split('\n',1)[0].replace('By','').split(',',1)[0].strip()

        except AttributeError:
            return auth   
    
    if data['hostname'] == 'www.bransontrilakesnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", { "itemprop" : "author" }).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return auth
    
    if data['hostname'] == 'dosmundos.com':
        auth = []
        try:
            soup = BeautifulSoup(article['text'], "html.parser")
            results=soup.find("div", {"class" : "elementor-element elementor-element-18433643 elementor-widget elementor-widget-theme-post-content"}).get_text()
            x = re.findall(r'(By\s\w+\s[A-Z].*?(?=[A-Z]))', results)
            auth.append(x)
            y = auth[0]
            return ' '.join(y).split('\n',1)[0].split('and',1)[0].replace('By','').strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['houstonherald.com', 'stljewishlight.org', 'www.marshallnews.com', 
                           'www.monett-times.com', 'mbcpathway.com', 'www.thepitchkc.com', 
                           'www.mymoinfo.com', 'whatsupkansascity.net', 'www.koamnewsnow.com']:
        auth= []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"twitter:data1"})
            auth.append(txt["content"].strip())
            return auth[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
    if data['hostname'] in ['www.missourinet.com']:
        auth= []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"twitter:data1"})
            auth.append(txt["content"].strip())
            return ' '.join(auth).split('@',1)[0].replace('.',' ').strip()
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        auth = []
        aut = data['meta_data']['twt-author-name']
        word_list = aut
        auth = word_list[3:]
        return auth
    
    if data['hostname'] == 'www.hannibal.net':
        auth = data['meta_data']['author']
        return auth

    else:
        
        author = data['author']
        return author

def news(item,data): 
    
    if data['hostname'] in ['molawyersmedia.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("meta", {"property" : "og:description"})
            nws2 = (results.get("content", None))
            nws.append(nws2) 
            return nws[0]
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['abc17news.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "entry__content"})
            for p in results.find_all('p'):
                if p.get_text().startswith('By'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.koamnewsnow.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "basic-content-wrap cf"})
            for p in results.find_all('p'):
                if p.get_text().startswith('\nvar'):
                    continue
                if p.get_text().startswith('COPYRIGHT'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.kmbc.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-content--body-text"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['fox2now.com', 'fox4kc.com', 'www.ozarksfirst.com', 'www.fourstateshomepage.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-content article-body rich-text"})
            for p in results.find_all('p'):
                if p.get_text().startswith('\nSubmit'):
                    continue
                if p.get_text().startswith('Δdocument'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.kctv5.com', 'www.kspr.com', 'www.kfvs12.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-body | article w-100"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.ksdk.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article__body"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''

    if data['hostname'] in ['www.kmov.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-body | article w-100"})
            for p in results.find_all('p', {"class" : "text"}):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['todaysfarmermagazine.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"property" : "text"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.417mag.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("section", {"class" : "story-detail padding-bottom-none"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.pulaskicountydaily.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("td", {"class" : "content_cell"})
            for p in results.find_all('p'):
                if p.get_text().startswith('Updated'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').replace("\\",'').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['morgancountypress.com', 'leader-statesman.com', 'eldoradospringsmo.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-padding clearfix"})
            for p in results.find_all('p'):
                if p.get_text().startswith('by'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').replace('\t','').encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['standard-democrat.com', 'www.semissourian.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "text"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.kahokamedia.com', 'www.themilanstandard.com', 'www.palmyra-spectator.com', 'www.waynecojournalbanner.com',
                           'www.shelbycountyherald.com', 'www.memphisdemocrat.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            for p in results.find_all('p'):
                if p.get_text().startswith('By'):
                    continue
                if p.get_text().startswith('If'):
                    continue
                if p.get_text().startswith(' Username'):
                    continue
                if p.get_text().startswith(' Password'):
                    continue
                if p.get_text().startswith('    Login'):
                    continue
                if p.get_text().startswith(' Lost'):
                    continue
                if p.get_text().startswith('Please'):
                    continue
                if p.get_text().startswith(' Email'):
                    continue
                if p.get_text().startswith(' Submit'):
                    continue
                if p.get_text().startswith('← Back'):
                    continue
                if p.get_text().startswith('Submitted'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['mdcp.nwaonline.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.thelickingnews.com', 'www.stegenherald.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            for p in results.find_all('p'):
                if p.get_text().startswith('Photos'):
                    continue
                if p.get_text().startswith('By'):
                    continue
                if p.get_text().startswith('Photo'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.lawrencecountyrecord.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "field field-name-body field-type-text-with-summary field-label-hidden"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').replace('\t','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['themissouritimes.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "post-content"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.warrensburgstarjournal.com', 'www.sedaliademocrat.com', 'www.westplainsdailyquill.net',
                           'www.fayettenewspapers.com', 'bolivarmonews.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"id" : "body"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['dosmundos.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "elementor-element elementor-element-18433643 elementor-widget elementor-widget-theme-post-content"})
            y = results.find('strong').get_text()
            z = re.sub(r'(By\s\w+\s[A-Z].*?(?=[A-Z]))', '', y)
            return ''.join(z)    
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.redlatinastl.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("section", {"class" : "blog_area single-post-area section-padding"})
            y = results.find_all('p')
            for z in y:
                nws.append(z.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')    
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['cedarrepublican.com', 'ccheadliner.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "body main-body clearfix"})
            x = results.find_all('p', {"class":"p1"})
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
            
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['bocojo.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "c5-white-box layout-2 c5-margin clearfix"})
            x = results.find_all('p')
            for y in x:
                if y.get_text().startswith('By:'):
                    continue
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').split('See more in', 1)[0].strip()
        
        except AttributeError:
            return ''
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.bethanyclipper.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "entry-content"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').split('How useful', 1)[0].strip()
        
        except AttributeError:
            return ''
        
    if data['hostname'] in ['www.laclederecord.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "body main-body clearfix"})
            x=results.get_text().encode('ascii', 'ignore').decode('ascii').split('<!', 1)[0].replace('\t','').replace('\n\n\n\n',' ').replace('\n','').strip()
            nws.append(x)
            return ' '.join(nws)
        
        
        except AttributeError:
            return ''
        except IndexError:
            return ''
    
    if data['hostname'] in ['moberlymonitor.com', 'gasconadecountyrepublican.com', 'www.kirksvilledailyexpress.com', 'mexicoledger.com', 
                           'mariescountyadvocate.com', 'marshfieldmail.com', 'www.mystandardnews.com', 'republicmonitor.com', 
                           'www.unterrifieddemocrat.com', 'www.warrencountyrecord.com', 'www.clintondailydemocrat.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "body main-body clearfix"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.dixonpilot.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','')
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['www.edinasentinel.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "text"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').replace('\n','').replace('If youre a current subscriber, log in below. If you would like to subscribe, please click the subscribe tab above.  Username    Password       Login   Lost your password?  Please enter your email and we will send your username and password to you.  Email    Submit   Back ','').strip()
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['dailyjournalonline.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            x = results.find_all('p')
            for y in x:
                nws.append(y.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split('  Get', 1)[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.ccscrnews.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for x in results:
                nws.append(x.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split("All content copyright", 1)[0].replace('Braden Clines','').replace('Kadee Brosseau DeCourley','').strip()
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'www.4bcaonline.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for x in results:
                nws.append(x.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii').split(" Barry County AdvertiserOffice:", 1)[0].replace('Adriana Keeton','')
            
        except AttributeError:
            return '' 
        
    if data['hostname'] in ['lewispnj.com', 'maconhomepress.com','www.theijnews.com', 'www.lamardemocrat.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "article-main"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.ktts.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "jeg-content-inner"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] in ['www.nevadadailymail.com', 'www.vandalialeader.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "content"})
            for p in results.find_all('p'):
                if p.get_text().startswith('By'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'themaneater.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "wp-container-2 wp-block-column"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
            
        except AttributeError:
            return '' 
    
    
    if data['hostname'] == 'plattecountylandmark.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "content-inner"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
            
        except AttributeError:
            return '' 
    
    
    if data['hostname'] in ['www.southeastarrow.com', 'www.theprospectnews.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"class" : "text"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
#.encode('ascii', 'ignore').decode('ascii') removes the \x91, etc. characters
            return ' '.join(nws).encode('ascii', 'ignore').decode('ascii')
            
        except AttributeError:
            return '' 
    
    if data['hostname'] == 'www.semissourian.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all('p')
            nws.append(results.get_text())
            return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for res in results:
                nws.append(res.text)
                return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'stlamerican.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        sep = 'TNCMS.AdManager.render'
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.split(sep, 1)[0])
                return nws[0]
        
        except AttributeError:
            return '' 

    if data['hostname'] == 'www.joplinglobe.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        sep = ';\n\n'
        sep2 = '\n\n\n'
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.split(sep, 1)[1].split(sep, 1)[1].split(sep2, 1)[0])
                return nws[0]
        
        except AttributeError:
            return '' 
    
    if data['hostname'] in ['www.hermannadvertisercourier.com','www.lakegazette.net', 'www.californiademocrat.com', 
                           'www.kq2.com', 'www.komu.com', 'www.mycouriertribune.com', 'www.myleaderpaper.com',
                           'www.lakenewsonline.com', 'www.lincolnnewsnow.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            for p in results.find_all('p'):
                if p.get_text().startswith('\nSuccess!'):
                    continue
                if p.get_text().startswith('\nError!'):
                    continue
                if p.get_text().startswith('Have the latest'):
                    continue
                if p.get_text().startswith('To report'):
                    continue
                nws.append(p.get_text())
            return ' '.join(nws).replace('\n','').encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''
    

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'www.columbiamissourian.com', 
                           'dailyjournalonline.com', 'www.maryvilleforum.com', 'websterkirkwoodtimes.com', 
                           'emissourian.com', 'www.phelpscountyfocus.com', 'www.webstercountycitizen.com',
                           'www.newstime-mo.com', 'www.nwmissourinews.com', 'www.hannibal.net']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.replace('\n','').encode('ascii', 'ignore').decode('ascii'))
                return nws[0]
        
        except AttributeError:
            return ''

    if data['hostname'] in ['the-standard.org']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0].encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return ''          


    if data['hostname'] == 'thesalemnewsonline.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeErrdeor:
            return ''
    
    
    if data['hostname'] == 'www.pikecountynews.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] in ['newspressnow.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find("div", {"itemprop" : "articleBody"})
            for p in results.find_all('p'):
                nws.append(p.get_text())
            return ' '.join(nws)
        
        except AttributeError:
            return ''
        
    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'blog-content'}).get_text()
            nws.append(txt)
            return nws[0]
        
        except AttributeError:
            return ''
        
    else:
        news = data['news']
        return news


def metadescription(item,data):
    if data['hostname'] == 'dosmundos.com':
        desc = []
        soup = BeautifulSoup(article['text'], "html.parser")
        descrip = soup.find("meta", property="og:description")
        des = descrip["content"]
        desc.append(des)
        return desc
    
def title(item,data):
    if data['hostname'] in ['www.firesideguard.com', 'www.theprospectnews.com', 'www.vandalialeader.com']:
        title = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('h1').get_text()
            title.append(txt)
            return title[0].encode('ascii', 'ignore').decode('ascii')
        
        except AttributeError:
            return '' 
        
    if data['hostname'] in ['www.plattechronicle.com']:
        title = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('span', attrs={'style':'font-size: 72px;'}).get_text()
            title.append(txt)
            return title[0].encode('ascii', 'ignore').decode('ascii').strip()
        
        except AttributeError:
            return '' 
    
def image(item,data):
    if data['hostname'] == 'www.pleasanthillview.com':
        image = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('section', attrs={'class':'article-content'})
            txt2 = txt.find('img')
            image.append(txt2['src'])
            return ' '.join(image).replace('/images', 'https://www.pleasanthillview.com/images')
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    

def source_1(item,data):
    if data['hostname'] == 'abc17news.com' and 'fox2now.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__user vcard author'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src

def process_article(item):
    article = Article(url=item["url"])
    article.download(input_html=item["text"])

    article.parse()
    article.nlp()
    url = urlparse(item["url"])
        # print(hashlib.sha256(item['url'].encode()).hexdigest)
    data = {
            "id": hashlib.sha256(item["url"].encode()).hexdigest(),
            "meta_data": article.meta_data,
            "title": str(article.title),
            "url": str(item["url"]),
            "news": str(article.text),
            "image": str(article.top_image),
            "date": str(article.publish_date),
            "author": article.authors,                                   # author derived from article.authors
            "hostname": url.hostname,
            "crawl_datetime": str(datetime.now()),
            "tags" : [],                                               # tags given by the outlet/reporter
            "inferred_tags_set1" : article.keywords,                 # tags derived from article.keywords
        
        }
    
    if len(data["news"]) > 10 and len(data["title"]) > 0:
        
        date = given_date(item,data)
        data['date'] = date
        # if date =='None':
        #     data['date'] = str(article.publish_date)
        
        tags = given_tags(item,data)
        data['tags'] = tags
 
        #src = source(item,data)
        #data['source'] = src
        
        #src = source_1(item,data)
        #data['source_1'] = src
        
        auth = author(item,data)
        data['author'] = auth
        
        nws = news(item,data)
        data['news'] = nws
    
        
    if len(data["title"]) == 0:
        headline = title(item,data)
        data['title'] = headline
        
    if data['hostname'] in ['www.pleasanthillview.com']:
        img = image(item,data)
        data['image'] = img

    if data['hostname'] in ['www.theprospectnews.com', 'www.vandalialeader.com', 'www.plattechronicle.com']:
        titl = title(item,data)
        data['title'] = titl
    
    if data['hostname'] in ['www.firesideguard.com', 'lewispnj.com']:
        date = given_date(item,data)
        data['date'] = date
        
    if data['hostname'] in ['lewispnj.com', 'maconhomepress.com', 'www.theijnews.com', 'www.lamardemocrat.com']:
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] in ['www.redlatinastl.com']:
        date = given_date(item,data)
        data['date'] = date
        
    if data['hostname'] in ['maconhomepress.com', 'www.theijnews.com', 'www.lamardemocrat.com']:
        auth = author(item,data)
        data['author'] = auth
        date = given_date(item,data)
        data['date'] = date
    
    if data['hostname'] == 'www.4bcaonline.com':
        nws = news(item,data)
        data['news'] = nws
        
        date = given_date(item,data)
        data['date'] = date
        
        auth = author(item,data)
        data['author'] = auth
        
    if data['hostname'] == 'bocojo.com':
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] == 'www.redlatinastl.com':
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] == 'mlawyersmedia.com':
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] in ['www.linncountyleader.com', 'myleaderpaper.com']:
        nws = news(item,data)
        data['news'] = nws
        
    if data['hostname'] in ['www.southmissouriannews.com', 'www.greenfieldvedette.com']:
        aut = author(item,data)
        data['author'] = aut
        
    if data['hostname'] in ['www.koamnewsnow.com']:
        aut = author(item,data)
        data['author'] = aut

    return data

In [17]:
curated_articles = []
for article in raw_articles: 
    curated_article = process_article(article)
    curated_articles.append(curated_article)

In [18]:
curated_articles

[{'id': '24eb10db45e82376e6a681e880e7cf8bbe2b12e34b342a2451b1412edf5a1ca4',
  'meta_data': defaultdict(dict,
              {'viewport': 'width=device-width, initial-scale=1, maximum-scale=1',
               'fb': {'app_id': 1782340335416902},
               'description': 'There really was a time when life seemed simple and not so complicated. In\xa0“Life Through the Seasons: A Story Collection,” John Comer reflects on those times. In his personal memoir, he describes life through the eyes of a farm boy growing up in rural Missouri during the 1940s and 1950s.',
               'og': {'site_name': 'West Plains Daily Quill',
                'description': 'There really was a time when life seemed simple and not so complicated. In\xa0“Life Through the Seasons: A Story Collection,” John Comer reflects on those times. In his personal memoir, he describes life through the eyes of a farm boy growing up in rural Missouri during the 1940s and 1950s.',
                'title': 'SUMMERSVILLE BOOKE